<a href="https://colab.research.google.com/github/rudraser/Sally/blob/main/ResNet_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [11]:
from matplotlib import pyplot as plt
import numpy as np
import collections


import torch.nn as nn

import torch.optim as optim

In [12]:
from torchvision import datasets, transforms
data_path = '/data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(size=[32,32], padding=4),                        
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [13]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [14]:
class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
            in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0, bias=False
        )
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False
        )
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels = 3, num_classes = 10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(
            block, layers[0], intermediate_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], intermediate_channels=128, stride=2
        )
        self.layer3 = self._make_layer(
            block, layers[2], intermediate_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], intermediate_channels=512, stride=2
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(
            block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)




In [15]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        
        print('{} Epoch {}, Training loss {}'.format(
            datetime.datetime.now(), epoch,
            loss_train / len(train_loader)))

In [16]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)

model =  ResNet(block,[3, 4, 23, 3]).to(device=device)# <1>
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [17]:
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list


(42520650,
 [9408,
  64,
  64,
  4096,
  64,
  64,
  36864,
  64,
  64,
  16384,
  256,
  256,
  16384,
  256,
  256,
  16384,
  64,
  64,
  36864,
  64,
  64,
  16384,
  256,
  256,
  16384,
  64,
  64,
  36864,
  64,
  64,
  16384,
  256,
  256,
  32768,
  128,
  128,
  147456,
  128,
  128,
  65536,
  512,
  512,
  131072,
  512,
  512,
  65536,
  128,
  128,
  147456,
  128,
  128,
  65536,
  512,
  512,
  65536,
  128,
  128,
  147456,
  128,
  128,
  65536,
  512,
  512,
  65536,
  128,
  128,
  147456,
  128,
  128,
  65536,
  512,
  512,
  131072,
  256,
  256,
  589824,
  256,
  256,
  262144,
  1024,
  1024,
  524288,
  1024,
  1024,
  262144,
  256,
  256,
  589824,
  256,
  256,
  262144,
  1024,
  1024,
  262144,
  256,
  256,
  589824,
  256,
  256,
  262144,
  1024,
  1024,
  262144,
  256,
  256,
  589824,
  256,
  256,
  262144,
  1024,
  1024,
  262144,
  256,
  256,
  589824,
  256,
  256,
  262144,
  1024,
  1024,
  262144,
  256,
  256,
  589824,
  256,
  256,
  26

In [18]:
training_loop(  
    n_epochs = 70,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2021-07-29 09:16:22.867839 Epoch 1, Training loss 2.1860355815619155
2021-07-29 09:17:58.987938 Epoch 2, Training loss 1.7510598006150913
2021-07-29 09:19:35.374013 Epoch 3, Training loss 1.5802017069228775
2021-07-29 09:21:12.392221 Epoch 4, Training loss 1.5094798551800916
2021-07-29 09:22:49.167851 Epoch 5, Training loss 1.4252638354173401
2021-07-29 09:24:26.135569 Epoch 6, Training loss 1.324174334249838
2021-07-29 09:26:03.186260 Epoch 7, Training loss 1.239900505832394
2021-07-29 09:27:39.637845 Epoch 8, Training loss 1.192122862695733
2021-07-29 09:29:16.468689 Epoch 9, Training loss 1.1114784996680287
2021-07-29 09:30:52.658065 Epoch 10, Training loss 1.042666361688653
2021-07-29 09:32:29.799340 Epoch 11, Training loss 1.005896872571667
2021-07-29 09:34:06.696465 Epoch 12, Training loss 0.9391157364906253
2021-07-29 09:35:43.232438 Epoch 13, Training loss 0.897639959478927
2021-07-29 09:37:20.193270 Epoch 14, Training loss 0.854370287388487
2021-07-29 09:38:56.407761 Epoch 15,

In [19]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.96
Accuracy val: 0.73
